In [1]:
import os
import shutil
import numpy as np

### CHANGE THE BASE DIRECTORY WITH YOUR CORRECT PATH OF THE FOLDER THAT CONTAINS ALL RESAMPLED DATA

In [ ]:
# Define the base directory containing patient folders
base_directory = r''

### UNIFY VALUES FOR TISSUE TYPES

- **0**: Background  
- **1**: Cavity  
- **2**: Normal Myocardium  
- **3**: Infarction  
- **4**: No-Reflow  


In [ ]:
# Create a mapping of original values to new values
value_mapping = {
    0: 0,       # Background
    500: 1,     # Cavity
    200: 2,     # Normal Myocardium
    1220: 3,    # Infarction
    2221: 2,    # Edema (Set it as normal myocardium)
    600: 0      # Right Ventricle (Set it as background)
}               # Value of 4 is the No-Reflow tissue


# Loop through each subfolder in the base directory
for subfolder_name in os.listdir(base_directory):
    subfolder_path = os.path.join(base_directory, subfolder_name)
    
    if os.path.isdir(subfolder_path):
        # Loop through each patient folder within the subfolder
        for folder_name in os.listdir(subfolder_path):
            folder_path = os.path.join(subfolder_path, folder_name)
            
            # Construct the path to the "Slices" folder
            slices_folder = os.path.join(folder_path, 'Slices')
            
            if os.path.exists(slices_folder) and os.path.isdir(slices_folder):
                # Loop through each .npy file in the Slices folder
                for npy_file in os.listdir(slices_folder):
                    if npy_file.endswith('.npy'):  # Only process .npy files
                        npy_file_path = os.path.join(slices_folder, npy_file)
                        
                        # Load the .npy file
                        np_array = np.load(npy_file_path)
                        
                        # Check if the array has the expected shape (height, width, 2)
                        if np_array.ndim == 3 and np_array.shape[2] == 2:
                            # Replace the values according to the mapping only on the contour channel (index 1)
                            contour_channel = np_array[:, :, 1]  # Extract the contour channel
                            
                            for original_value, new_value in value_mapping.items():
                                contour_channel[contour_channel == original_value] = new_value
                            
                            # Assign the modified contour channel back to the array
                            np_array[:, :, 1] = contour_channel
                        
                            # Save the modified array back to the same file or a new file
                            np.save(npy_file_path, np_array)
                            
                            print(f'Processed and saved: {npy_file_path}')
                        else:
                            print(f'Warning: {npy_file_path} does not have the expected shape (height, width, 2).')

### ADD DATASET PREFIX TO THE SLICE NAME

In [ ]:
# Loop through each subfolder in the base directory
for subfolder_name in os.listdir(base_directory):
    subfolder_path = os.path.join(base_directory, subfolder_name)
    
    if os.path.isdir(subfolder_path):  # Check if it's a directory
        # Determine the prefix based on the subfolder name
        if subfolder_name.startswith("EMIDEC"):
            prefix = "EMIDEC"
        elif subfolder_name.startswith("MyoPS"):
            prefix = "MyoPS"
        else:
            print(f"Skipping subfolder: {subfolder_name} (No valid prefix found)")
            continue  # Skip if the prefix cannot be determined
        
        # Loop through each patient folder within the subfolder
        for folder_name in os.listdir(subfolder_path):
            folder_path = os.path.join(subfolder_path, folder_name)
            
            # Construct the path to the "Slices" folder
            slices_folder = os.path.join(folder_path, 'Slices')
            
            if os.path.exists(slices_folder) and os.path.isdir(slices_folder):
                # Loop through each .npy file in the Slices folder
                for npy_file in os.listdir(slices_folder):
                    if npy_file.endswith('.npy'):  # Only process .npy files
                        npy_file_path = os.path.join(slices_folder, npy_file)
                        
                        # Create the new filename with the prefix
                        new_file_name = f'{prefix}_{npy_file}'
                        new_file_path = os.path.join(slices_folder, new_file_name)

                        # Rename the file
                        os.rename(npy_file_path, new_file_path)
                        
                        print(f'Renamed: {npy_file_path} to {new_file_path}')


### COPY ALL TRAIN AND TEST SLICES TO COMBINE THEM AT TWO FOLDERS IN THE SAME BASE DIRECTORY 

In [3]:
# Define target directories for Train and Test
train_target_directory = os.path.join(base_directory, 'All Train Slices')
test_target_directory = os.path.join(base_directory, 'All Test Slices')

# Create target directories if they don't exist
os.makedirs(train_target_directory, exist_ok=True)
os.makedirs(test_target_directory, exist_ok=True)

# Loop through each subfolder in the base directory
for subfolder_name in os.listdir(base_directory):
    subfolder_path = os.path.join(base_directory, subfolder_name)

    # Check if the subfolder name ends with "Train" or "Test"
    if os.path.isdir(subfolder_path) and (subfolder_name.endswith('Train') or subfolder_name.endswith('Test')):
        # Determine the target directory based on the subfolder name
        target_directory = train_target_directory if subfolder_name.endswith('Train') else test_target_directory
        
        # Loop through each patient folder in the subfolder
        for patient_folder in os.listdir(subfolder_path):
            patient_path = os.path.join(subfolder_path, patient_folder)

            # Construct the path to the "Slices" folder
            slices_folder = os.path.join(patient_path, 'Slices')

            if os.path.exists(slices_folder) and os.path.isdir(slices_folder):
                # Loop through each .npy file in the Slices folder
                for npy_file in os.listdir(slices_folder):
                    if npy_file.endswith('.npy'):  # Only process .npy files
                        npy_file_path = os.path.join(slices_folder, npy_file)
                        
                        # Copy the .npy file to the target directory
                        shutil.copy(npy_file_path, target_directory)
                        print(f'Copied: {npy_file_path} to {target_directory}')

print('All NPY files have been copied successfully.')

Copied: D:\Studying\Fourth year\GP\Datasets\Standardized Dataset\All Data Resampled and Unified\EMIDEC_Test\Case_101\Slices\EMIDEC_Case_101_slice_1.npy to D:\Studying\Fourth year\GP\Datasets\Standardized Dataset\All Data Resampled and Unified\All Test Slices
Copied: D:\Studying\Fourth year\GP\Datasets\Standardized Dataset\All Data Resampled and Unified\EMIDEC_Test\Case_101\Slices\EMIDEC_Case_101_slice_2.npy to D:\Studying\Fourth year\GP\Datasets\Standardized Dataset\All Data Resampled and Unified\All Test Slices
Copied: D:\Studying\Fourth year\GP\Datasets\Standardized Dataset\All Data Resampled and Unified\EMIDEC_Test\Case_101\Slices\EMIDEC_Case_101_slice_3.npy to D:\Studying\Fourth year\GP\Datasets\Standardized Dataset\All Data Resampled and Unified\All Test Slices
Copied: D:\Studying\Fourth year\GP\Datasets\Standardized Dataset\All Data Resampled and Unified\EMIDEC_Test\Case_101\Slices\EMIDEC_Case_101_slice_4.npy to D:\Studying\Fourth year\GP\Datasets\Standardized Dataset\All Data Re